In [1]:
import os
import midas_touch2 as mt2
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import stock_mapping as sm
import warnings
import asyncio
import twap
import time
import json
import concurrent.futures
from datetime import datetime, timedelta
from class_mapping import StockSymbolMapper
pd.options.display.max_columns = 100000
warnings.filterwarnings('ignore')

if not mt5.initialize():
    print("initialize() failed")
else:
    print("MT5 successfully initialised.\n")

MT5 successfully initialised.



In [2]:
current_dir = os.getcwd()
#current_path = os.path.abspath(__file__)
#current_dir = os.path.dirname(current_path)
config_file_name = 'config.json'
config_file_path = os.path.join(current_dir, config_file_name)
with open(config_file_path, 'r') as config_file:
        config = json.load(config_file)

symbols = config['symbols']
api_token = config['api_token']
mapping_file = config['mapping_file_path']
sleep_time = config['sleep_time']
account=config['execute_account']
password=config['password']
server=config['server']
VaR = config['VaR']
starting_cash = config['starting_cash']

In [3]:
authorized=mt5.login(account, password=password, server=server)
if authorized:
    print(mt5.account_info())
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))


AccountInfo(login=2117, trade_mode=0, leverage=1, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=1, currency_digits=2, fifo_close=False, balance=999878529.36, credit=0.0, profit=-2768.0, equity=999878529.36, margin=30380.4, margin_free=999848148.96, margin_level=3291196.0650946004, margin_so_call=50.0, margin_so_so=30.0, margin_initial=30380.4, margin_maintenance=30380.4, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Jeff Ng', server='KIBB-Prod', currency='MYR', company='Kenanga Investment Bank Berhad')


In [4]:
mt5_symbol = sm.get_ticker_list(symbols, file_path=mapping_file)
symbols = [symbol + "SE" for symbol in symbols]
price_df = mt2.download_stocks(symbols, "d", api_token, num_days=20)
price_df = pd.DataFrame(price_df)
price_df.columns = mt5_symbol
weights_df, latest_weights = mt2.run_portfolio(price_df, alpha_n=5)
print(f'latest_weights: {latest_weights}')
#ADD DUMMY WEIGHTS HERE TO TEST
total_investment = starting_cash
previous_allocation_df = mt2.get_folder(folder_name='daily_allocation')
previous_portfolio_df = mt2.get_folder(folder_name='daily_portfolio')

latest_weights: HLFG      -0.000000
PBBANK     0.000000
FFB        0.000000
MAYBANK    0.293706
MSM        0.000000
HARTA      0.706294
Name: 2023-12-01, dtype: float64


In [5]:
if previous_allocation_df is not None and previous_portfolio_df is not None:
    prev_holding = previous_allocation_df['Holding Units']
    prev_entry = previous_allocation_df['Entry Units']
    prev_exit = previous_allocation_df['Exit Units']
    total_investment = abs(previous_portfolio_df['Total Value'].iloc[-1])
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    allocation_df['Holding Units'] = prev_holding + prev_entry - prev_exit
else: #first run
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    print("This is the initial run. No previous allocation data available.")
    allocation_df['Holding Units'] = 0

# Calculate Entry and Exit Units
allocation_df['Entry Units'] = allocation_df.apply(
    lambda row: max(row['Allocated Units'] - row['Holding Units'], 0), axis=1)
allocation_df['Exit Units'] = allocation_df.apply(
    lambda row: max(row['Holding Units'] - row['Allocated Units'], 0), axis=1)
numeric_cols = allocation_df.select_dtypes(include=['number']).astype(float)
for col in numeric_cols.columns:
    allocation_df[col] = numeric_cols[col]
print(allocation_df)

  Share Symbol  Share Price  Allocated Units   Weights  Holding Units  \
0         HLFG        16.48              0.0 -0.000000          400.0   
1       PBBANK         4.32              0.0  0.000000         1400.0   
2          FFB         1.39              0.0  0.000000        11400.0   
3      MAYBANK         9.00           3200.0  0.293706         2600.0   
4          MSM         1.46              0.0  0.000000         9400.0   
5        HARTA         2.46          28700.0  0.706294        13300.0   

   Entry Units  Exit Units  
0          0.0       400.0  
1          0.0      1400.0  
2          0.0     11400.0  
3        600.0         0.0  
4          0.0      9400.0  
5      15400.0         0.0  


In [6]:

order_ids = mt2.execute_trades_from_data(allocation_df)

Placing exit order for HLFG
The corresponding side of the order book does not exist to continue operation. Stop execution.
Placing exit order for PBBANK
The corresponding side of the order book does not exist to continue operation. Stop execution.
Placing exit order for FFB
The corresponding side of the order book does not exist to continue operation. Stop execution.
Placing entry order for MAYBANK
The corresponding side of the order book does not exist to continue operation. Stop execution.
Placing exit order for MSM
The corresponding side of the order book does not exist to continue operation. Stop execution.
Placing entry order for HARTA
The corresponding side of the order book does not exist to continue operation. Stop execution.


In [7]:
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,HLFG,16.48,0.0,-0.000000,400.0,0.0,400.0
1,PBBANK,4.32,0.0,0.000000,1400.0,0.0,1400.0
2,FFB,1.39,0.0,0.000000,11400.0,0.0,11400.0
3,MAYBANK,9.00,3200.0,0.293706,2600.0,600.0,0.0
4,MSM,1.46,0.0,0.000000,9400.0,0.0,9400.0
5,HARTA,2.46,28700.0,0.706294,13300.0,15400.0,0.0
